In [134]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer, Image, PageBreak

In [36]:
df = pd.read_excel('data.xlsx', header=1)
df['Quiz Perf.'] = df['Quiz Perf.'].apply(lambda x: round(float(str(x).strip('%')) * 100, 2))

In [64]:
topics = df.columns[4:]
print(topics)
df.head()

Index(['Uncertainties', 'Units', 'Change in velocity', '1D Motion',
       '2D Motion', 'Momentum', 'N Laws', 'Equilibrium', 'Work Done',
       'Basics of CM', 'Horizontal CM', 'Orbital Motion',
       'Gravitational Potential', 'G-Field strength', 'SHM'],
      dtype='object')


,Unnamed: 0,Quiz submission,Quiz Perf.,Attempted qns,Uncertainties,Units,Change in velocity,1D Motion,2D Motion,Momentum,N Laws,Equilibrium,Work Done,Basics of CM,Horizontal CM,Orbital Motion,Gravitational Potential,G-Field strength,SHM
0,Student#1,1.000000,70.11,174,7/9,4/10,3/5,15/18,8/11,17/22,4/8,11/11,6/8,5/6,3/4,5/10,2/3,6/6,6/12
1,Student#2,1.000000,78.74,174,8/9,10/10,3/5,16/18,8/11,18/22,8/8,7/11,6/8,6/6,4/4,7/10,1/3,5/6,6/12
2,Student#3,0.551724,66.67,96,6/7,6/6,3/4,8/12,4/8,6/11,2/3,5/5,2/3,0/2,2/3,4/6,0/1,1/2,4/7
3,Student#4,0.540230,71.28,94,3/6,6/7,3/3,4/11,5/6,9/13,6/6,6/8,4/7,3/3,0/0,1/2,0/1,1/2,2/3
4,Student#5,0.810345,48.94,141,5/8,8/9,0/5,6/16,6/11,7/19,1/8,7/9,2/7,3/5,1/1,4/5,1/3,3/5,3/8


In [151]:
def create_radar_chart(student_data, student_name):
    scores = [student_data[topic] for topic in topics]
    percentages = [int(s.split('/')[0]) / int(s.split('/')[1])
                   if int(s.split('/')[1]) != 0 else 0 for s in scores]

    angles = np.linspace(0, 2 * np.pi, len(topics), endpoint=False).tolist()
    percentages += percentages[:1]
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.fill(angles, percentages, color='blue', alpha=0.25)
    ax.plot(angles, percentages, color='blue', linewidth=2)

    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(topics, fontsize=10)

    chart_file = f"{student_name}_radar_chart.png"
    plt.savefig(chart_file)
    plt.close()

    return chart_file

In [152]:
def generate_feedback(student_data):
    feedback = []
    topics_to_improve = []
    topics_to_praise = []
    
    for topic in topics:
        score = student_data[topic].split('/')[0]
        try:
            score_percentage = int(score) / int(student_data[topic].split('/')[1]) * 100
        except ZeroDivisionError:
            score_percentage = 0

        if score_percentage < 50:
            topics_to_improve.append(topic)
        elif score_percentage > 80:
            topics_to_praise.append(topic)
    
    if topics_to_improve:
        feedback.append(f"You should improve on the following topics: {', '.join(topics_to_improve)}.")
    if topics_to_praise:
        feedback.append(f"Good job on the following topics: {', '.join(topics_to_praise)}.")
    
    if not topics_to_improve and not topics_to_praise:
        feedback.append("Your performance is consistent across all topics. Keep up the good work!")
    
    return "<br/>".join(feedback)

In [153]:
def generate_pdf_report(student_data, student_name):
    file_name = f'{student_name}_Physics_Quiz_Report.pdf'
    document = SimpleDocTemplate(file_name, pagesize=A4, rightMargin=30, leftMargin=30, topMargin=30, bottomMargin=18)
    styles = getSampleStyleSheet()

    title_style = ParagraphStyle(name="TitleStyle", fontSize=18, spaceAfter=20)
    title = Paragraph(f"Physics Quiz Report: {student_name}", title_style)

    general_info_data = [
        ['Quiz Performance', f"{student_data['Quiz Perf.']}%"],
        ['Attempted Questions', f"{student_data['Attempted qns']}"]
    ]

    general_info_table = Table(general_info_data, colWidths=[2.5*inch, 2.5*inch], hAlign='LEFT')
    general_info_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.lightblue),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
    ]))

    data = [['Topic', 'Score']] + [[topic, f"{student_data[topic]}"] for topic in topics]
    
    scores_table = Table(data, colWidths=[2.5*inch, 2.5*inch], hAlign='LEFT')
    scores_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
    ]))
     
    feedback_text = generate_feedback(student_data)
    
    radar_chart_file = create_radar_chart(student_data, student_name)
    radar_chart_img = Image(radar_chart_file, 4.5*inch, 4.5*inch)
    
    elements = [
        title,
        Spacer(1, 12),
        general_info_table,
        Spacer(1, 12),
        Paragraph('Detailed Topic Scores', styles['Heading2']),
        scores_table,
        Spacer(1, 12),
        Paragraph('Qualitative Feedback', styles['Heading2']),
        Paragraph(feedback_text, styles['Normal']),
        PageBreak(),
        Paragraph('Radar Chart', styles['Heading2']),
        radar_chart_img,
    ]
    document.build(elements)

In [154]:
for index, row in df.iterrows():
    student_name = f"Student#{index + 1}"
    generate_pdf_report(row, student_name)